In [1]:
import pandas as pd
import os
import sys
from pandarallel import pandarallel
import numpy as np

In [2]:
pandarallel.initialize()

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 2 workers


Change `REPO_NAME` to whatever you named the Git repo.

In [3]:
HOME_FOLDER = os.path.expanduser("~")
REPO_NAME = "cs325b-airquality"
REPO_FOLDER = os.path.join(HOME_FOLDER, REPO_NAME)

In [4]:
print("Assumes Git repo is ", REPO_FOLDER)

Assumes Git repo is  /home/robbiejones/cs325b-airquality


In [5]:
sys.path.append(REPO_FOLDER)
import utils

Change `DATA_FOLDER` below to the appropriate folder storing the gcloud bucket.
This module assumes the bucket data is stored in `~/${DATA_FOLDER}`, with EPA readings stored in `~/${DATA_FOLDER}/epa` and Sentinel data stored in `~/${DATA_FOLDER}/sentinel`.

In [6]:
DATA_FOLDER = "es262-airquality"
EPA_DATA_FOLDER = os.path.join(HOME_FOLDER, DATA_FOLDER, "epa")
SENTINEL_FOLDER = os.path.join(HOME_FOLDER, DATA_FOLDER, "sentinel")
SENTINEL_METADATA_FOLDER = os.path.join(SENTINEL_FOLDER, "Metadata")

Run the cell below and verify the folder paths are correct!

In [7]:
print("Will look for EPA data in ", EPA_DATA_FOLDER)
print("Will look for Sentinel data in ", SENTINEL_FOLDER)

Will look for EPA data in  /home/robbiejones/es262-airquality/epa
Will look for Sentinel data in  /home/robbiejones/es262-airquality/sentinel


In [8]:
%%time
# Can take a while if running for the first time
utils.rename_all_sentinel_files(SENTINEL_FOLDER)

CPU times: user 128 ms, sys: 28 ms, total: 156 ms
Wall time: 157 ms


In [9]:
%%time
# load all EPA data into a DataFrame (can take up to a minute or so)
epa_df = utils.load_csv_dfs(EPA_DATA_FOLDER)

CPU times: user 4.71 s, sys: 716 ms, total: 5.43 s
Wall time: 5.43 s


In [10]:
%%time
# can take anywhere from 10-15 minutes.
dates = utils.load_sentinel_dates(SENTINEL_METADATA_FOLDER)

CPU times: user 4min 12s, sys: 4.55 s, total: 4min 17s
Wall time: 12min 17s


In [11]:
%%time
# can take anywhere from 10-15 minutes.
new_df = epa_df.assign(SENTINEL_FILENAME = "", SENTINEL_INDEX = -1)
del epa_df
new_df = new_df.parallel_apply(utils.add_sentinel_info, axis=1, metadata_folder_path=SENTINEL_METADATA_FOLDER, sentinel_dates = dates)

CPU times: user 9.75 s, sys: 2.08 s, total: 11.8 s
Wall time: 10min 11s


In [12]:
del dates

In [13]:
%%time
new_df.to_csv(os.path.join(EPA_DATA_FOLDER, "combined") + ".csv")

CPU times: user 23.7 s, sys: 368 ms, total: 24.1 s
Wall time: 24 s
